# Exercises Day04

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


## Ex01

**Estimate an integral $\int_0^1 e^x d x$ by simulation (the Crude Monte Carlo estimator). Use e.g. an estimator based on 100 samples and present the result as the point estimator and a confidence interval.**

## Ex02

**Estimate an integral $\int_0^1 e^x d x$ by simulation (the Crude Monte Carlo estimator). Use e.g. an estimator based on 100 samples and present.**